# CropsDB V0.2

## CRoissance Optimale des Plantes en Sol

Colecte des donnees environementales pour unbe croissance optimale des plantes
et stockage de ces donnees dans une base de donmnee de type couchDB.

la colecte des donnees repose sur l'utilisation d'un ESP32 equipe des capteurs suivants:
 - Le capteur DHT22 permet la mesure de la temperature ambiante
   ainsi que du niveau d'humiditee ambiante
 - Le capteur CSMS V1.2 permet la mesure de la quantitee d'eau dans le sol.
   CSMS: (Capacitive Soil Moisture Sensor)  
 - Le captteur BMP085 permet la mesure de la pression et de la temperature
   (de maniere semblet'il plus precise aue le DHT mais je ne l'ai pas verifie)
 - Le capteur TSL2561 Permet de mesurer la lumiere recue par les plantes


Ce notebook Jupyter permet d'analyser les donnees captees par les differents capteurs et stookees dans la base CouchDB. Il est a notter qu le maximum de donnees disponibles sont stokees et analysees et pas seulement les donees purement relatives aux plantes. C'est important car ceci permet de mieux comprendre d'eventueles dysfonctionement du systeme.

 
Auteur: kolergy{@}gmail.com

## Explications du Code

les imports permettent de faire apel aux libraries necessaires pour l'analyse:
 - couchdb: permet de se coonecter ave la base de donnees du meme nom et de stocker ou lire des donnees.
 - pandas: est une librairie de traitement des donnees et permet de manipuler aisement de grandes quantitee de donnees.
 - matplotlib: permet de generer simplement des graphes, a noter que seule la partie pyplot de cette librarie est chargee. 

In [ ]:
import couchdb
import pandas as pd
import matplotlib.pyplot as plt

Dans cette section on definit les parametres a modifier par l'utilisateur, ici les parametres de connection a la base de donnee CouchDB. 
Puisd on procede a la conection a celle-ci.

In [ ]:
DBUser      = "DBUsername"
DBPass      = "DBPassword"
DBAdr       = "DBAdress"
DBPort      = "5984"         # DB Port
couchserver = couchdb.Server("http://%s:%s@%s:%s/" % (DBUser, DBPass,DBAdr,DBPort))

Permet de lister l'ensemble des bases existantes sur le serveur

In [ ]:
for dbname in couchserver:
    print(dbname)

Si la base "crops" est presente sur le serveur on s'y connecte grace a la variable db, si celle-ci n'existep pas on la cree

In [ ]:
dbname = "crops"
if dbname in couchserver:
    db = couchserver[dbname]
else:
    db = couchserver.create(dbname)

Une fois connecte a la base de donnees crops on copie l'integralitee de son contenu dans un dataframe de la bibliotheque Pandas que l'on nomme df

In [ ]:
rows = db.view('_all_docs', include_docs=True)
data = [row['doc'] for row in rows]
df   = pd.DataFrame(data)

On peut imprimer les noms de colones pour identifier les parametre interessant, il est possible de lister les types de donnees associes a chaque colones 

In [ ]:
print(df.columns)
#df.dtypes

Dans le programe de l'ESP le marquage temporel ie: l'horodatage est fait par le stockage de la date dans une variable et de l'heure dans une autre pour etre en mesure d'indexer proprement nos donees il est necessaire de combiner ces deux valeurs afin d'avoir un marquer temporel complet

In [ ]:
df['Period'] = df.Date.str.cat(df.Hour,sep=" ")
#print(df.Period)

- Ce marquer d'horodatage complet est transforme en objet datetime qui permet aux pandas d'effectuer des operations temporelles.
- Ensuite cet horodatage dans la forme adequate est utilise come index de notre dataframe.
- Pour le prouver on afiche les dernieres lignes du dataframe

In [ ]:
df['date']   = pd.to_datetime(df['Period'])
df.index     = df['date']
df.tail()


La on s'apercoit que les dernieres valeurs ne sont pas les bonnes, en effet on a atribue un nouvel index a la base de donnees, mais celle-ci n'as pas ete triee par raport a celui-ci.
- Il faut donc effectuer un tri sur l'index
- on peut ensuite verifier que cela a fonctione

Ce tri n'est pas necessaire pour permetre la bone generation de graphes mair permet de mieux s'y retrouver quant on veux faire des operations sur la data frame

In [ ]:
df.sort_index(inplace=True)
df.tail()

- On stoke la derniere base en CSV cela permet d'utiliser d'autres outils d'analyse (a eviter pour de tres grosses bases)

- puis on regarde si il y as des enregistrementds dans le futur: si il y en a il ne faut pas conclure que le voyage dans le temps est possible mais plustot certains enregistreemts on des information temporelles erronees.

In [ ]:
df.to_csv("MyData.csv")

now     = pd.Timestamp.now()

df[df['date'] > now]

In [ ]:

oneDay  = now-pd.Timedelta('1 day')
oneWeek = now-pd.Timedelta('7 day')
oneyear = now-pd.Timedelta('365 day')

df[df['date'] < oneyear]


In [ ]:
now     = pd.Timestamp.now()
df = df.drop(df[df['date'] > now].index)
df = df.drop(df[df['date'] < oneyear].index)

In [ ]:
ax =  df.Temp.plot(figsize=(15, 10))
df.Hum.plot(secondary_y=True, style='g')
ax.set_ylabel('Temperature °C')
ax.right_ax.set_ylabel('Humiditee %')

In [ ]:
print(pd.Timedelta('1 day'))
print(pd.Timestamp.now()-pd.Timedelta('1 day'))

df["2019/04/16"].date.tail()

In [ ]:
ax =  df[df['date'] > oneWeek].Temp.plot(figsize=(15, 10))
df[df['date'] > oneWeek].Hum.plot(secondary_y=True, style='g')
ax.set_ylabel('Temperature °C')
ax.right_ax.set_ylabel('Humiditee %')

In [ ]:
ax =  df[df['date'] > oneDay].Temp.plot(figsize=(15, 10))
df[df['date'] > oneDay].Hum.plot(secondary_y=True, style='g')
ax.set_ylabel('Temperature °C')
ax.right_ax.set_ylabel('Humiditee %')

In [ ]:
ax =  df.plot(y=['Water1','Water2'], figsize=(15, 10))

In [ ]:
df.plot(y=['Water1%','Water2%'], figsize=(15, 10))

In [ ]:
df[df['date'] > oneDay].plot(y=['Water1%','Water2%'], figsize=(15, 10))

In [ ]:
df.Epoch.plot(figsize=(15, 10))

In [ ]:
df.date.plot(figsize=(15, 10))

In [ ]:
df.plot(y=['tSet','tWif','tNTP', 'tJso'], figsize=(15, 10))

In [ ]:
df.plot(y=['tt0'], figsize=(15, 10))

In [ ]:
df.RSSI.plot(figsize=(15, 10))

In [ ]:
df.DTim.plot(figsize=(15, 10))

In [ ]:
df.UpTime.plot(figsize=(15, 10))

In [ ]:
df.plot(y=['Pin[0]:','Pin[2]:','Pin[12]:','Pin[13]:','Pin[14]:','Pin[15]:','Pin[25]:','Pin[26]:','Pin[27]:'], figsize=(15, 10))


In [ ]:
df.plot(y=['Pin[32]:','Pin[33]:','Pin[36]:','Pin[39]:'], figsize=(15, 10))